# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

## Get Required Files from Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
import warnings
import tensorflow as tf
import pickle
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import pprint
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Dropout,BatchNormalization,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from numpy import array
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
tf.__version__

'2.2.0'

In [0]:
#Set your project path 
project_path = "/content/drive/My Drive/"


#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [0]:
data_path="/content/drive/My Drive/Sarcasm Detection/Data/"

In [0]:
df=pd.read_json("/content/drive/My Drive/Sarcasm Detection/Data/Sarcasm_Headlines_Dataset.json",lines=True )

In [8]:
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [0]:
df1=df.drop(labels='article_link', axis=1,inplace=True)


In [10]:
df.head(2)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0


In [11]:
df.shape

(26709, 2)

In [12]:
df.dtypes

headline        object
is_sarcastic     int64
dtype: object

In [13]:
df['is_sarcastic'].value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [14]:
headline_max_length=max(len(txt) for txt in df['headline'])
headline_max_length

254

In [15]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['headline']) #Fit it on Headlines
input_seq = tokenizer.texts_to_sequences(df['headline'])
input_seq[0:2] #Display some converted sentences

[[307, 15114, 678, 3336, 2297, 47, 381, 2575, 15115, 5, 2576, 8433],
 [3, 8434, 3337, 2745, 21, 1, 165, 8435, 415, 3111, 5, 257, 8, 1001]]

In [16]:
vocab_size = len(tokenizer.word_index)
print(vocab_size)

29656


In [17]:
input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, 
                                                                   maxlen=headline_max_length,
                                                                   padding='pre')
print(input_seq.shape)

(26709, 254)


#**## Modelling**

## Import required modules required for modelling.

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [0]:
max_features = 10000
maxlen = headline_max_length
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

# Define X and y for your model.

In [20]:
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0  

In [0]:
X=input_seq
y=np.asarray(df['is_sarcastic'])


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2, 
    random_state=42
)

In [23]:
y_val.shape

(5342,)

In [24]:
X_train.shape

(21367, 254)

## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [26]:
vocab_size = len(tokenizer.word_index)
print(vocab_size)

29656


#**## Word Embedding**

## Get Glove Word Embeddings

In [0]:
glove_file = data_path + "glove.6B.zip"

In [0]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [0]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd



# Create a weight matrix for words in training docs

In [30]:
embedding_matrix = np.zeros((vocab_size+1, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [0]:
### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))

In [0]:
model = tf.keras.Sequential()

In [0]:
model.add(tf.keras.layers.Embedding(input_dim=vocab_size+1,
                  output_dim=embedding_size,
                  weights=[embedding_matrix],
                  trainable=False, mask_zero= True))

In [0]:
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(LSTM(units=128,return_sequences=False),merge_mode='concat'))


In [38]:
model.output

<tf.Tensor 'bidirectional/Identity:0' shape=(None, 256) dtype=float32>

In [0]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         5931400   
_________________________________________________________________
dropout (Dropout)            (None, None, 200)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               336896    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 6,268,553
Trainable params: 337,153
Non-trainable params: 5,931,400
_________________________________________________________________


# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [0]:
batch_size = 100
epochs = 5

## Add your code here ##

In [43]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=100, validation_data=(X_val, y_val))

Epoch 1/5
214/214 [==============================] - 324s 2s/step - loss: 0.4921 - accuracy: 0.7584 - val_loss: 0.3962 - val_accuracy: 0.8192
Epoch 2/5
214/214 [==============================] - 322s 2s/step - loss: 0.3610 - accuracy: 0.8369 - val_loss: 0.3418 - val_accuracy: 0.8484
Epoch 3/5
214/214 [==============================] - 321s 2s/step - loss: 0.2995 - accuracy: 0.8703 - val_loss: 0.3026 - val_accuracy: 0.8718
Epoch 4/5
214/214 [==============================] - 318s 1s/step - loss: 0.2561 - accuracy: 0.8930 - val_loss: 0.3023 - val_accuracy: 0.8706
Epoch 5/5
214/214 [==============================] - 316s 1s/step - loss: 0.2184 - accuracy: 0.9114 - val_loss: 0.2944 - val_accuracy: 0.8774


Validation Accuracy with Embedding size of 200 and RNN Units of 128 is 87.74

# Model 2- Glove Embedding of 300, With 256 Memory Size

In [0]:
EMBEDDING_FILE = './glove.6B.300d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

In [36]:
embedding_matrix = np.zeros((vocab_size+1, 300))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

In [0]:
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(input_dim=vocab_size+1,
                  output_dim=300,
                  weights=[embedding_matrix],
                  trainable=False, mask_zero= True))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Bidirectional(LSTM(units=256,return_sequences=False),merge_mode='concat'))
model2.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
model2.summary()

In [38]:
model2.fit(X_train,y_train,
          epochs=5,
          batch_size=100, validation_data=(X_val, y_val))

Epoch 1/5
214/214 [==============================] - 220s 1s/step - loss: 0.4546 - accuracy: 0.7835 - val_loss: 0.3662 - val_accuracy: 0.8371
Epoch 2/5
214/214 [==============================] - 220s 1s/step - loss: 0.3299 - accuracy: 0.8539 - val_loss: 0.3260 - val_accuracy: 0.8551
Epoch 3/5
214/214 [==============================] - 220s 1s/step - loss: 0.2610 - accuracy: 0.8905 - val_loss: 0.3036 - val_accuracy: 0.8678
Epoch 4/5
214/214 [==============================] - 218s 1s/step - loss: 0.2098 - accuracy: 0.9156 - val_loss: 0.2934 - val_accuracy: 0.8761
Epoch 5/5
214/214 [==============================] - 217s 1s/step - loss: 0.1552 - accuracy: 0.9389 - val_loss: 0.3134 - val_accuracy: 0.8757


Validation Accuracy with Embedding size of 300 and RNN Units of 256 is 87.57

# Model 3- Glove Embedding of 300, With 128 Memory Size

In [0]:
model3 = tf.keras.Sequential()
model3.add(tf.keras.layers.Embedding(input_dim=vocab_size+1,
                  output_dim=300,
                  weights=[embedding_matrix],
                  trainable=False, mask_zero= True))
model3.add(tf.keras.layers.Dropout(0.2))
model3.add(tf.keras.layers.Bidirectional(LSTM(units=128,return_sequences=False),merge_mode='concat'))
model3.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [42]:
model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 300)         8897100   
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               439296    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 9,336,653
Trainable params: 439,553
Non-trainable params: 8,897,100
_________________________________________________________________


In [45]:
model3.fit(X_train,y_train,
          epochs=15,
          batch_size=100, validation_data=(X_val, y_val))

Epoch 1/15
214/214 [==============================] - 216s 1s/step - loss: 0.1345 - accuracy: 0.9509 - val_loss: 0.3294 - val_accuracy: 0.8720
Epoch 2/15
214/214 [==============================] - 214s 1000ms/step - loss: 0.1037 - accuracy: 0.9620 - val_loss: 0.3336 - val_accuracy: 0.8808
Epoch 3/15
214/214 [==============================] - 214s 1s/step - loss: 0.0829 - accuracy: 0.9701 - val_loss: 0.3479 - val_accuracy: 0.8819
Epoch 4/15
214/214 [==============================] - 213s 994ms/step - loss: 0.0632 - accuracy: 0.9786 - val_loss: 0.3914 - val_accuracy: 0.8778
Epoch 5/15
214/214 [==============================] - 213s 993ms/step - loss: 0.0489 - accuracy: 0.9834 - val_loss: 0.3836 - val_accuracy: 0.8815
Epoch 6/15
214/214 [==============================] - 214s 1s/step - loss: 0.0355 - accuracy: 0.9887 - val_loss: 0.4166 - val_accuracy: 0.8871
Epoch 7/15
214/214 [==============================] - 214s 999ms/step - loss: 0.0323 - accuracy: 0.9897 - val_loss: 0.4527 - val_acc

Validation Accuracy with Embedding size of 300 and RNN Units of 128 is 87.46